Oulu_NLPTM_TwitterBrexit Data Collection and Preprocessing

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Julian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Julian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import os
import tweepy as tw
import pandas as pd

import re
import string
from unidecode import unidecode

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import PorterStemmer

import sys
sys.path.insert(0, '../config')

from twitter_dev_access import *


# Global Parameters
stop_words = set(stopwords.words('english'))

# amount of tweets collected is tweets_per_page * no_of_pages 
# note that retweets will be directly removed, but are still considered to this number
tweets_per_page = 100 # max 200
no_of_pages = 12

In [3]:
# access twitter
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)


In [4]:
conservatives = ['@BorisJohnson', '@theresa_may']
conservative_house_of_commons = ['@nadams', '@BimAfolami', '@AdamAfriyie', '@imranahmadkhan', '@peter_aldous', '@lucyallan', '@amessd_southend', '@Stuart4WolvesSW', '@Caroline_Ansell', '@AthertonNWales', '@GarethBaconMP', '@richardbaconmp', '@KemiBadenoch', '@ShaunBaileyUK', '@Siobhan_Baillie']
conservatives.extend(conservative_house_of_commons)
labour = ['@HackneyAbbott', '@Debbie_abrahams', '@rushanaraali', '@TahirAliMP', '@DrRosena', '@MikeAmesburyMP', '@PutneyFleur', '@ToniaAntoniazzi', '@JonAshworth', '@PaulaBarkerMP', '@ApsanaBegumMP', '@hilarybennmp', '@_OliviaBlake', '@PaulBlomfieldMP', '@TracyBrabin', '@BenPBradshaw', '@KevinBrennanMP', '@lynbrownmp']

In [5]:
def collect_tweets(twitterusers):
    tweet_list = []

    for n, twitter_name in enumerate(twitterusers):
        # get tweets for user, exlude retweets and get full tweet content
        pages = tw.Cursor(api.user_timeline, screen_name=twitter_name, include_rts=False, tweet_mode="extended", count=tweets_per_page).pages(no_of_pages)
        
        # get tweets from pages
        tweet_texts = [tweet.full_text for page in pages for tweet in page][:1000]

        print('(' + str(n+1) + '/' + str(len(twitterusers)) + ') Collected ' + str(len(tweet_texts)) + ' tweets from ' + twitter_name)

        # only add newest 100 tweets to tweet list
        tweet_list.extend(tweet_texts)
    
    print('All tweets collected for this party: '+ str(len(tweet_list)))
    
    return tweet_list

In [6]:
def clean(tweet):
    # from https://www.pluralsight.com/guides/building-a-twitter-sentiment-analysis-in-python
    tweet = tweet.lower()
    
    # Remove unicode characters (emotes etc)
    tweet = unidecode(tweet)
    
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#','', tweet)
    # Remove html encodings like &amp;
    tweet = re.sub(r'&\w+;','', tweet)
    # Remove punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    
    # Remove stopwords
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    
    # use stemmer
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
    
    return " ".join(stemmed_words)
    

In [7]:
def store(tweet_list, filename):
    with open(filename, 'w') as f:
        for tweet in tweet_list:
            f.write("%s\n" % tweet)
    print(f"saved tweets to {filename}")

In [8]:
conservatives_tweets = collect_tweets(conservatives)

(1/17) Collected 872 tweets from @BorisJohnson
(2/17) Collected 682 tweets from @theresa_may
(3/17) Collected 664 tweets from @nadams
(4/17) Collected 320 tweets from @BimAfolami
(5/17) Collected 1000 tweets from @AdamAfriyie
(6/17) Collected 371 tweets from @imranahmadkhan
(7/17) Collected 817 tweets from @peter_aldous
(8/17) Collected 683 tweets from @lucyallan
(9/17) Collected 997 tweets from @amessd_southend
(10/17) Collected 653 tweets from @Stuart4WolvesSW
(11/17) Collected 570 tweets from @Caroline_Ansell
(12/17) Collected 878 tweets from @AthertonNWales
(13/17) Collected 247 tweets from @GarethBaconMP
(14/17) Collected 23 tweets from @richardbaconmp
(15/17) Collected 471 tweets from @KemiBadenoch
(16/17) Collected 937 tweets from @ShaunBaileyUK
(17/17) Collected 430 tweets from @Siobhan_Baillie
All tweets collected for this party: 10615


In [9]:
cleaned_conservative = [clean(tweet) for tweet in conservatives_tweets]

In [10]:
store(cleaned_conservative, "conservative_tweets_preprocessed.txt")

saved tweets to conservative_tweets_preprocessed.txt


In [11]:
labour_tweets = collect_tweets(labour)

(1/18) Collected 748 tweets from @HackneyAbbott
(2/18) Collected 628 tweets from @Debbie_abrahams
(3/18) Collected 239 tweets from @rushanaraali
(4/18) Collected 217 tweets from @TahirAliMP
(5/18) Collected 722 tweets from @DrRosena
(6/18) Collected 214 tweets from @MikeAmesburyMP
(7/18) Collected 719 tweets from @PutneyFleur
(8/18) Collected 499 tweets from @ToniaAntoniazzi
(9/18) Collected 487 tweets from @JonAshworth
(10/18) Collected 409 tweets from @PaulaBarkerMP
(11/18) Collected 473 tweets from @ApsanaBegumMP
(12/18) Collected 464 tweets from @hilarybennmp
(13/18) Collected 552 tweets from @_OliviaBlake
(14/18) Collected 787 tweets from @PaulBlomfieldMP
(15/18) Collected 662 tweets from @TracyBrabin
(16/18) Collected 687 tweets from @BenPBradshaw
(17/18) Collected 396 tweets from @KevinBrennanMP
(18/18) Collected 102 tweets from @lynbrownmp
All tweets collected for this party: 9005


In [12]:
cleaned_labour = [clean(tweet) for tweet in labour_tweets]

In [13]:
store(cleaned_labour, "labour_tweets_preprocessed.txt")

saved tweets to labour_tweets_preprocessed.txt


0. watch live updat coronaviru 20 octob 2020
1. congratul win second term new zealand pm work togeth tackl climat chang forg excit new trade partnership uk nz great thing look forward futur
2. next week new rapid covid19 test triall across countri enabl us test nh care home staff much frequent
3. watch live updat coronaviru 16 octob 2020
4. updat negoti eu
5. good meet today alway support fantast farmer work tirelessli pandem keep countri fed outsid eu chanc get better fairer deal britain food farm sector
6. simplifi standardis local rule england introduc three local covid alert level level accompani proportion restrict find local covid alert level area postcod checker
7. watch live updat coronaviru 12 octob 2020
8. real risk lost gener girl never return school let happen global recoveri pandem must leav one behind dayofthegirl urg global action stand everi girl right 12 year qualiti educ
9. today recognis local often unsung hero whose hard work dedic help carri us pandem care compass 

1045. wish christian countri around world bless goodfriday
1046. want wish jewish famili gather around seder tabl even happi peac pesach today chanc celebr enorm last contribut made jewish commun uk everi walk life passov
1047. must protect improv environ inherit went us respons pass inherit next gener
1048. en solidarit avec la franc et le francai le cloch de labbay de westminst sonneront 18h43 heur de pari ce soir pour marquer linstant ou le feu sest declar hier
1049. underlin solidar franc peopl bell westminst abbey toll 543pm even mark moment fire began yesterday
1050. thought peopl franc tonight emerg servic fight terribl blaze notredam cathedr
1051. 18 million worker benefit biggest ever increas nationallivingwag month protect lowestpaid worker help build economi work everyon
1052. act protect rent privat sector unfair evict give tenant longterm certainti peac mind deserv
1053. wish commun celebr vaisakhi happi peac time reflect valu truth toler compass hope renew
1054. ban degra

1226. mani congratul chosen candid next gener elect know youll excel job work repres local area
1227. fun afternoon great question pupil burchett green maidenhead help award impress st nichola primari hurst
1228. realli enjoy answer question morn
1229. believ everi fibr cours set right one countri peopl
1230. deal deliv brexit british peopl vote
1231. firmli believ head heart decis best interest entir unit kingdom
1232. uniqu strength ingenu forg global futur countri everi bit excit anyth come togeth secur futur prosper gener come
1233. one hundr year gun fell silent western front everi one us paus reflect immens sacrific made mani armisticeday100
1234. proud repres countri thiepval presid today battlefield franc scar horror war import moment reflect friend partner europ
1235. st symphorien honour lay wreath grave john parr georg ellison first last uk soldier die war rememb everi member arm forc gave live protect hold dear never forget
1236. great open new factori thatcham today meet m

1786. 50000 brit return australia alon last month
1787. idiot tweet someon who never job real world
1788. thank graduat uk govern cheven programm fundrais effort total ship 800 ventil china seek deliveri
1789. sincer condol famili staff member kill myanmar transport covid19 sampl yet anoth remind ceasefir need rakhin face challeng covid19 critic aid worker full access protect
1790. news uk govern charter flight home manila
1791. info govt charter flight back pakistan follow see detail
1792. love cover
1793. thank pleasur construct discuss progress updat help brit return home
1794. hope find explan use bull stori peopl appear fallen
1795. team across asia region 100 focus help british peopl return home updat work far latest advic still need return uk pleas check travel advic
1796. may educ know plenti clever halfwit appear one
1797. spot gaffer
1798. time extraordinari challeng may wish everyon bless easter sunday stayhomestaysaf
1799. spot
1800. bori got power thank great
1801. havent 

2672. spot rishi flow research innov idea underpin uk futur job prosper place world budget2020
2673. exactli right approach chancellor nh must get what need deal coronaviru budget2020
2674. solid open budget chancellor ps30bn fiscal stimulu see small busi worker lowest paid coronaviru much might need short term budget2020
2675. longterm solut adult social care mean must look polici make sensibl decis im glad govern share view meet minist urg boost adult social care precept may well way forward
2676. good serv deleg legisl committe today increas child tax credit disabl benefit children least well household year ahead ensur household gain increas incom moretodo
2677. sadden attend funer wonder dee quick st stephen st agn warm kind fun yet principl profession offic thought john famili goodby nice ladi mayor children would say ill miss
2678. look forward strengthen tie ghana develop nation new independ trade polici im glad govern agre globalbritain
2679. wonder morn beauti rich biodivers c

...

7915. carersweek join thank carer fantast work wrexham across uk unpaid carer provid lifelin mani commun behalf everyon wrexham thank
7916. id also like thank gather peac wrexham weekend adher social distanc guidelin
7917. statement blacklivesmatt protest
7918. absolut right reassess huawei place 5g network china recent sabrerattl reaffirm belief propos involv economi curtail
7919. glimps ive wrexham week
7920. alway grate courag alway rememb grave sacrific made name freedom human dday76 lestweforget
7921. today 76th anniversari dday join think brave men land beach normandi 76 year ago fought die free evil fascism
7922. england scotland made move toward dentist return work urg
7923. letter offer full support creat crimin offenc trespass
7924. mani wrexham busi alreadi benefit busi rate grant scheme make sure dont miss applic close 30th june inform appli
7925. 6yearold jimmi run mile everi day he run length marathon saveourzoo
7926. biggest chariti zoo uk resid wrexham grate doorstep co

7997. today firefightersmemorialday recognis braveri sacrific firefight lost live whilst protect commun fmd2020
7998. fantast video resourc avail teacher parent suitabl home school classroom environ primari pupil secondari student
7999. commemor 75th anniversari veday75 parliament curat materi archiv visit explor key legisl famou speech stori reconstruct bomb damag
8000. check latest beyondthebubbl podcast
8001. 122346 test carri yesterday surpass govern target well done everyon help achiev
8002. congratul nomin award excel work member volunt tenni vote jonathon
8003. take part favouritef answer question celebr import preciou green space favourit childhood park favourit local park favourit oversea park favourit park memori favourit thing park
8004. record first ever podcast last night discuss everyth effect covid19 wrexham power ballad
8005. colleagu call upon ensur covid19 test avail wale match rest uk peopl wale must left behind suffer
8006. must develop addit test site across wale e

8746. caller right iopc need signific reform
8747. great see former colleagu citi hall make progress nation stage receiv promot make huge contribut countri
8748. 
8749. close 35 fire station clearli dont faintest idea your talk
8750. excel choic great job citi hall make brilliant mp
8751. founder chair incompar relat today present lya annual recommend mayor london citi hall lta would exist without kati head univers leav thrive lya good hand
8752. huge turnout final conserv parti leadership hust backbori
8753. context ps78 million equat 130 polic offic entir wrong prioriti nicetohav essenti get grip london ever increas violent crime epidem far import
8754. pleas see wonder pennoy centr award ps68700 grant govern cultur recoveri fund well done team pennoy hard work especi difficult time
8755. back boriss deal fulfil promis leav eu perfect dump backstop crucial peopl want leav deal focu instead school fund nh cut crime hous getbrexitdon
8756. hard sure qualiti anyth suprem court judgement

8874. excel respons cant overemphasis much preciou time tori mp spend rebut fake news orchestr campaign instead serv constitu send mani sort letter everi week
8875. weird pl follow ill send dm instead definit correct address think know bounc back
8876. youv still got detail sent mep 2015 experi happi share id love read guarante would use could come use futur debat email profil thank
8877. ha sure ive got permiss third would need consult variou stakehold husband whip etc
8878. wasnt end final debat start around 105am home 2am deleg legisl later morn 855amand babi boy wake 542 like clockwork
8879. pl stop retweet propaganda malevol distort polit gain zac say best
8880. good econom news paper today ahead budget boe say pay rise unemploy fall cbi survey show strongest manufactur order 30 yr export order 22yr firm confid growth continu
8881. union came assembl dont want happen
8882. peopl said sameth dockland light railway go happen tim
8883. dear essex follow say get extra invest essex pol

9644. chanc better citi becom opportun rout poverti rout crime know youth worker 20 year give peopl opportun thrive end day want thing
9645. 8pm im debat race here ill say believ equal opportun reason behind london success equal isnt defund polic end capit unlock potenti everi london black white
9646. 
9647. youth worker 20 year ive stop search hundr time ill 8pm talk legaci stephen lawrenc whether britain chang issu shook london uk effect still felt
9648. long way weve come far im confid
9649. thank andrew
9650. 1993 teenag call stephen lawrenc murder black 27 year later see black live matter protest statu toppl histori scrutinis itv want know britain chang tune tomorrow 8pm hear ive got say
9651. spoke polic cut sadiq khan play polit peopl live safeti mayor side law order shouldnt defund polic agre sign petit
9652. said
9653. 
9654. charg polic london announc plan cut polic budget ps110m that 1692 offic street london deserv feel safe agre sign petit
9655. know id prefer
9656. need ge